In [1]:
# os.chdir('..')
import split_video_gpx; reload(split_video_gpx)
from split_video_gpx import *

import utils; reload(utils)
from utils import *

In [10]:
os.chdir('Sample Data/')
print os.getcwd()
gpx_files = glob.glob("GPX/*.gpx")
gpx_video_match = []
parse_result = []
bad_quality_thres=10
stop_thres=0.05
short_stop=10
smooth=True
largest_chunk_seconds = 30

for gpx_f in gpx_files:
    # extract information
    video_name, lon_lats, timestamps, duration = parse_gpx(gpx_f)
    # quality control
    has_video = os.path.isfile(video_name)
    is_bad_max = bad_quality_max_diff(timestamps, thres=bad_quality_thres)
    is_bad_int = bad_quality_diff_not_int(timestamps)
    gpx_video_match.append((gpx_f, video_name, has_video, is_bad_max, is_bad_int))

    # skip parsing the trace, 
    # if the video for a gpx can't be found, or the quality of of gpx is bad
    if not has_video or is_bad_max or is_bad_int:
        continue
    
    # preprocess locations and timestamps
    # fill in timestamps gap
    fill_gpx, fill_timestamps = fill_gpx_gap(lon_lats, timestamps)
    # alignment based on video duration
    vdur_round = int(round(duration_of_video(video_name)))
    aligned_gpx, aligned_timestamps = align_loc_ts_duration(fill_gpx, fill_timestamps,vdur_round)
    aligned = zip(range(len(aligned_gpx)), aligned_gpx, aligned_timestamps)
    
    # find separate points by long stop
    split_points = split_points_indices_by_stop(aligned_gpx, stop_thres=stop_thres, short_stop=short_stop, smooth=smooth)

    # get chunks based on long stop and largest chunk second size
    chunks = []
    for i, j in split_points:
        chunks.extend(list(even_chunks(aligned[i:j+1], largest_chunk_seconds)))
    
    parse_result.append((gpx_f, lon_lats, timestamps, duration_of_video(video_name),
                         vdur_round, len(lon_lats), len(timestamps), len(aligned_gpx), split_points, chunks))
    
os.chdir('..')

/mnt/d/★★学习工作/Life in Maryland/Research Project：CyclingSafe/Cycling_safety/video_gpx/Sample Data


In [11]:
df_match = pd.DataFrame(gpx_video_match, columns=['gpx', 'vfile', 'match', 'bad_max', 'bad_int'])


In [12]:
df_videos = pd.DataFrame(parse_result, columns=['gpx','lon_lats', 'timestamps', 'vdur', 'vdur_round', 'len_locs', 'len_ts',
                                               'len_aligned', 'split_points', 'chunks'])

In [329]:
trace = df_videos.loc[7]
print df_match[df_match.gpx==trace.gpx].vfile

locs = trace.lon_lats
lat_lons = [(lat, lon) for lon, lat in locs]

dict_locs = {l:i for i, l in enumerate(list(set(locs)))}
id2loc = {i:l for l, i in dict_locs.items()}
locs_id = [dict_locs[l] for l in locs]

ts = trace.timestamps

pd.DataFrame([locs_id, ts]).T.to_csv('a.csv', sep='\t')
new_locs, new_ts = fill_gpx_gap(locs, ts)
new_locs_id = [dict_locs[l] for l in new_locs]
new_lat_lons = [(lat, lon) for lon, lat in new_locs]
pd.DataFrame([new_locs_id, new_ts, new]).T.to_csv('b.csv', sep='\t')

12    DCIM/104_VIRB/VIRB0004.MP4
Name: vfile, dtype: object


In [105]:
# vis the trace

import geopandas as gp
from shapely.geometry import Point


from leaflet_creation import create_map_visualization
gpdf = gp.GeoDataFrame(
    zip(lat_lons, locs_id, ts, [Point(x) for x in locs]), columns=['lonlat', 'id','ts','geometry']).reset_index()
gpdf.geometry = gpdf.buffer(0.000005)
gpdf['color'] = '#aa0'


html_title = 'vis trace'
file_path = ''
file_name = 'vis trace'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets', 'stsg']
map_layers = ['light','streets', 'satellite']
binding_data=[['ly', 'trace']]
gpdfs = [gpdf]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)